In [1]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
import time

In [2]:
!nvidia-smi

Thu Oct  3 14:13:51 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

In [3]:
gpt2.download_gpt2(model_name="355M")

Fetching checkpoint: 1.05Mit [00:00, 278Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 62.6Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 664Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:15, 92.6Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 265Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 57.2Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 135Mit/s]                                                       


In [4]:
file_name = "../input/LesMiserables.txt"

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

**EXTRA IMPORTANT NOTE**: If running the notebook in Google Colab, it is advised to mount your Drive to be able to save checkpoints and re-start from where you stopped in case the notebook crashes or times out. 

In [ ]:
gpt2.mount_gdrive()

In [5]:
sess = gpt2.start_tf_sess()

In [6]:
%%time
gpt2.finetune(sess,
              dataset=file_name,
              model_name='355M',
              steps=2000,
              restore_from='fresh', #change to 'latest' if restarting fine-tuning from saved checkpoint
              checkpoint_dir = "drive/My Drive/gpt2/checkpoint", #if wanting to save checkpoints on Drive
              run_name='run1',
              print_every=10,
              learning_rate=1e-4,
              sample_every=500, 
              save_every=1000, #if the fine-tuning is very slow, lower this number 
              overwrite=True)

Loading checkpoint models/355M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:11<00:00, 11.83s/it]


dataset has 1203667 tokens
Training...
[10 | 24.03] loss=3.25 avg=3.25
[20 | 32.75] loss=3.31 avg=3.28
[30 | 41.40] loss=3.33 avg=3.29
[40 | 50.10] loss=3.19 avg=3.27
[50 | 58.78] loss=3.05 avg=3.22
[60 | 67.47] loss=3.07 avg=3.20
[70 | 76.14] loss=3.48 avg=3.24
[80 | 84.84] loss=3.25 avg=3.24
[90 | 93.55] loss=3.51 avg=3.27
[100 | 102.22] loss=3.31 avg=3.28
[110 | 110.93] loss=3.07 avg=3.26
[120 | 119.64] loss=2.94 avg=3.23
[130 | 128.31] loss=2.94 avg=3.20
[140 | 136.99] loss=2.96 avg=3.19
[150 | 145.69] loss=3.07 avg=3.18
[160 | 154.39] loss=2.88 avg=3.16
[170 | 163.08] loss=2.89 avg=3.14
[180 | 171.77] loss=3.12 avg=3.14
[190 | 180.42] loss=2.81 avg=3.12
[200 | 189.09] loss=2.97 avg=3.11
[210 | 197.78] loss=3.13 avg=3.11
[220 | 206.44] loss=3.12 avg=3.11
[230 | 215.38] loss=2.80 avg=3.10
[240 | 224.04] loss=3.07 avg=3.10
[250 | 232.70] loss=2.83 avg=3.09
[260 | 241.39] loss=3.20 avg=3.09
[270 | 250.06] loss=2.86 avg=3.08
[280 | 258.76] loss=3.12 avg=3.08
[290 | 267.44] loss=2.40 av

[1040 | 976.85] loss=2.06 avg=2.71
[1050 | 985.50] loss=2.64 avg=2.71
[1060 | 994.14] loss=3.15 avg=2.72
[1070 | 1002.85] loss=2.65 avg=2.71
[1080 | 1011.51] loss=2.66 avg=2.71
[1090 | 1020.18] loss=2.76 avg=2.71
[1100 | 1028.88] loss=2.80 avg=2.72
[1110 | 1037.57] loss=1.84 avg=2.70
[1120 | 1046.25] loss=2.09 avg=2.69
[1130 | 1054.96] loss=2.63 avg=2.69
[1140 | 1063.63] loss=2.24 avg=2.69
[1150 | 1072.28] loss=2.78 avg=2.69
[1160 | 1080.96] loss=2.42 avg=2.68
[1170 | 1089.64] loss=2.70 avg=2.68
[1180 | 1098.31] loss=2.98 avg=2.69
[1190 | 1106.95] loss=2.61 avg=2.69
[1200 | 1115.63] loss=2.45 avg=2.68
[1210 | 1124.46] loss=2.70 avg=2.68
[1220 | 1133.12] loss=2.44 avg=2.68
[1230 | 1141.80] loss=2.15 avg=2.67
[1240 | 1150.45] loss=2.87 avg=2.68
[1250 | 1159.16] loss=2.65 avg=2.68
[1260 | 1167.86] loss=2.70 avg=2.68
[1270 | 1176.54] loss=2.72 avg=2.68
[1280 | 1185.20] loss=2.47 avg=2.67
[1290 | 1193.90] loss=2.85 avg=2.68
[1300 | 1202.58] loss=2.88 avg=2.68
[1310 | 1211.22] loss=2.54 avg=

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [7]:
gpt2.reset_session(sess)
del sess
sess = gpt2.start_tf_sess()

In [8]:
%%time
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-2000
CPU times: user 10.5 s, sys: 512 ms, total: 11 s
Wall time: 10.9 s


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [10]:
gpt2.generate(sess, run_name='run1')

 jusqu’à ce que je sois fait, il ne le comprend pas.

Il était dit, parmi les gens qui les avait réfractées, que je peux. J’en sais qu’il le soir, et que je peux.

Le soir est une sorte de joie que vous devez toujours échappées. Il se réveille, et il vous repose.

Le soir, je t’ai vu l’élément qui passe.

C’est que je peux, il n’y a rien d’étourdissement.

Le soir, je le sais.

Sur ce pauvre chose, j’ai pris l’épanouissement.

J’en suis, je vous laisse.

Le soir, je le sais.

Quelqu’un qui veut attendre, c’est que je peux.

Cela, c’est, pour le couvent, que je n’en veux pas ici. Il est mort.

Quand il est mort, je le sais.

Je le sais.

J’ai pourtant qu’on peut être mort d’être le moment où je ne vois pas sortir de cet homme pour sortir de la vie.

Je n’en veux pas.

Je le sais.

Je le sais.

Je le sais.

Je le sais.

Je le sais.

Je le sais.

Je le sais.

Je le sais.

Je le sais.

Je le sais.

Je le sais.

Je le sais.

Je le sais.

Je le sais.

Vous me faites l’ignominie.

J’en sais, 

In [ ]:
%%time
text = gpt2.generate(sess, return_as_list=True)[0]

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [16]:
gpt2.generate(sess,
              length=500,
              temperature=0.7,
              top_k=50,
              prefix="- Monsieur")

- Monsieur les deux autres, et dans cet homme, l’aspiration de l’enfant, le bonheur de la mère, vous, monsieur, vous, je n’ai pas besoin de la lumière pour vous, monsieur, je n’ai pas besoin de la lumière pour vous. Je vous le dire que je ne verrai que de vous, et que je vous connaît. Je vous ai vécu les ritements qui sont difformes, les soixante jouissements, les ténèbres, les gonds de l’enfant, les choses de la vie, les mères du moi, les étoiles et les couverts de la vie, le soir, le voile de l’enfant, et les choses de l’aspiration, le bonheur de la mère.

Il poursuivit : – Je vous m’appelle, en ce cas. Je vous ai donc à vous. Je vous ai dit à vous. Il y a des jouissances toujours où je pense que je vous ai dit. Il y a des hommes qui sont des hommes, et je vous ai réunissé. C’est des gens qui ont les âmes de la vie. J’ai de la voir une voiture de jouissances, parce que vous alliez de la vie, c’est un grand trot jouissant et sévère.

C’était une sorte d’arrête de lui qui le voyait bie

In [17]:
!du -h checkpoint/

1.4G	checkpoint/run1
1.4G	checkpoint/


For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [0]:
# may have to run twice to get file to download
files.download(gen_file)

# Download model:

In [20]:
!tar czvf ./model-lesmiserables.tar.gz checkpoint/run1/ 

checkpoint/run1/
checkpoint/run1/model-2000.meta
checkpoint/run1/model-2000.data-00000-of-00001
checkpoint/run1/counter
checkpoint/run1/encoder.json
checkpoint/run1/vocab.bpe
checkpoint/run1/checkpoint
checkpoint/run1/hparams.json
checkpoint/run1/model-2000.index
checkpoint/run1/events.out.tfevents.1570112199.b225f5908fdf


In [21]:
!

__notebook_source__.ipynb  model-lesmiserables.tar.gz  samples
checkpoint		   models


<a href="./model-lesmiserables.tar.gz">Download File</a>
